**TUGAS PRAKTIKUM 2**

Dengan menggunakan data yang sama, buatlah ensemble dengan algoritma 
1. Logistic Regression
2. SVM Polynomial
3. Decission Tree

Anda boleh melakukan eksplorasi dengan melakukan tunning hyperparameter

Import Library

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.svm import SVC # import SVM classifier
from sklearn.tree import DecisionTreeClassifier #import DT
from sklearn.ensemble import VotingClassifier # import model Voting
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

Persiapan Data

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

#Load data
dbt = pd.read_csv('/content/drive/My Drive/Colab Notebooks/diabetes.csv')

dbt.head()

Mounted at /content/drive


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
#Cek nama kolom
dbt.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [4]:
#Cek kolom null
dbt.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose','BloodPlessure' ataupun 'Insulin'
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

#Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
#Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
  print("============================================")
  print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


In [6]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer (missing_values=0, strategy="mean", copy=False)
dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns])

Split data training dan testing

In [7]:
X = dbt[feature_columns]
y = dbt.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Training dengan Logistic Regression

Standarisasi Fitur

In [8]:
# Proses standarisasi agar data terdistribusi secara normal,

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

Training dan Evaluasi

In [9]:
#Buat obyek Logistic Regression
lg_std = LogisticRegression()

# Fit dengan data yang telah di standarisasi
lg_std.fit (X_train_std, y_train)

#Prediksi dengan data test
y_pred_lg = lg_std.predict(X_test_std)

#Evaluasi akurasi testing data
acc_lg = accuracy_score(y_test, y_pred_lg)

#Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_lg))
print(f"Test set accuracy: {acc_lg}")

Test set accuracy: 0.74
Test set accuracy: 0.7359307359307359


Training dengan SVM Polynomial

In [10]:
#Model SVM liner tanpa tunnning hyperparameter
svm_pol = SVC(kernel='poly', degree=8)

#Fit ke model
svm_pol.fit(X_train_std, y_train)

#Prediksi
y_pred_svm_pol = svm_pol.predict(X_test_std)

#Evaluasi akurasi testing data
acc_svm_pol = accuracy_score(y_test, y_pred_svm_pol)

#Print hasil evaluasi
print("Test set accuracy: {:.2f}" .format(acc_svm_pol))
print(f"Test set accuracy: {acc_svm_pol}")


Test set accuracy: 0.67
Test set accuracy: 0.6666666666666666


Training dengan Decission Tree

In [11]:
#Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
#Pada kasus kali ini kita akan menggunakan parameter default
ndt = DecisionTreeClassifier()

#Fit ke model
ndt.fit(X_train_std, y_train)

#Prediksi
y_pred_ndt = ndt.predict(X_test_std)

#Evaluasi akurasi testing data
acc_ndt = accuracy_score(y_test, y_pred_ndt)

#Print hasil evaluasi
print("Test set accuracy: {:.2f}" .format(acc_ndt))
print(f"Test set accuracy: {acc_ndt}")

Test set accuracy: 0.71
Test set accuracy: 0.7142857142857143


Training dengan Voting

In [12]:
#Definisikan algoritma yang akan digunakan untuk voting

clfa=LogisticRegression()
clfb=SVC(kernel='poly')
clfc=DecisionTreeClassifier()

# model hard voting
voting = VotingClassifier(estimators=[('LogisticRegression', clfa), ('SVM-POL', clfb), ('DecisionTree', clfc)], voting='hard')

#Fit model
voting.fit(X_train_std, y_train)

# Prediksi
y_pred_vta = voting.predict (X_test_std)

#Evaluasi akurasi testing data
acc_vta = accuracy_score(y_test, y_pred_vta)

# Print hasil evaluasi
print ('Voting Hard')
print ("Test set accuracy: {:.2f}".format(acc_vta))
print(f"Test set accuracy: {acc_vta}")

Voting Hard
Test set accuracy: 0.73
Test set accuracy: 0.7316017316017316
